# Импорт библиотек

In [1]:
import os
import warnings
import pandas as pd
import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.utils import compute_sample_weight
from sklearn.metrics import roc_curve, precision_recall_curve, classification_report, roc_auc_score

warnings.filterwarnings('ignore')

# Чтение исходных данных

In [2]:
# путь до данных на компьютере
path = 'G:/ds-junior/final_work_ds/train_data'

In [3]:
def read_parquet_dataset_from_local(path_to_dataset: str, start_from: int = 0,
                                    num_parts_to_read: int = 12, columns=None, verbose=False) -> pd.DataFrame:
    """
    читает num_parts_to_read партиций, преобразовывает их к pd.DataFrame и возвращает
    :param path_to_dataset: путь до директории с партициями
    :param start_from: номер партиции, с которой нужно начать чтение
    :param num_parts_to_read: количество партиций, которые требуется прочитать
    :param columns: список колонок, которые нужно прочитать из партиции
    :return: pd.DataFrame
    """

    res = []
    dataset_paths = sorted([os.path.join(path_to_dataset, filename) for filename in os.listdir(path_to_dataset)
                            if filename.startswith('train')])
    print(dataset_paths)

    start_from = max(0, start_from)
    chunks = dataset_paths[start_from: start_from + num_parts_to_read]
    if verbose:
        print('Reading chunks:\n')
        for chunk in chunks:
            print(chunk)
    for chunk_path in tqdm.tqdm_notebook(chunks, desc="Reading dataset with pandas"):
        print('chunk_path', chunk_path)
        chunk = pd.read_parquet(chunk_path,columns=columns)
        res.append(chunk)

    return pd.concat(res).reset_index(drop=True)

In [4]:
def prepare_transactions_dataset(path_to_dataset: str, num_parts_to_preprocess_at_once: int = 1, num_parts_total: int=50,
                                 save_to_path=None, verbose: bool=False):
    """
    возвращает готовый pd.DataFrame с признаками, на которых можно учить модель для целевой задачи
    path_to_dataset: str
        путь до датасета с партициями
    num_parts_to_preprocess_at_once: int
        количество партиций, которые будут одновременно держаться и обрабатываться в памяти
    num_parts_total: int
        общее количество партиций, которые нужно обработать
    save_to_path: str
        путь до папки, в которой будет сохранён каждый обработанный блок в .parquet-формате; если None, то не будет сохранён
    verbose: bool
        логирует каждую обрабатываемую часть данных
    """
    preprocessed_frames = []

    for step in tqdm.tqdm_notebook(range(0, num_parts_total, num_parts_to_preprocess_at_once),
                                   desc="Transforming transactions data"):
        transactions_frame = read_parquet_dataset_from_local(path_to_dataset, step, num_parts_to_preprocess_at_once,
                                                             verbose=verbose)


        #здесь должен быть препроцессинг данных


        #записываем подготовленные данные в файл
        if save_to_path:
            block_as_str = str(step)
            if len(block_as_str) == 1:
                block_as_str = '00' + block_as_str
            else:
                block_as_str = '0' + block_as_str
            transactions_frame.to_parquet(os.path.join(save_to_path, f'processed_chunk_{block_as_str}.parquet'))

        preprocessed_frames.append(transactions_frame)
    return pd.concat(preprocessed_frames)

In [5]:
data = prepare_transactions_dataset(path, num_parts_to_preprocess_at_once=2, num_parts_total=12,
                                    save_to_path='G:/ds-junior/final_work_ds/train_data')

Transforming transactions data:   0%|          | 0/6 [00:00<?, ?it/s]

['G:/ds-junior/final_work_ds/train_data\\train_data_0.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_1.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_10.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_11.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_2.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_3.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_4.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_5.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_6.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_7.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_8.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_9.pq', 'G:/ds-junior/final_work_ds/train_data\\train_target.csv']


Reading dataset with pandas:   0%|          | 0/2 [00:00<?, ?it/s]

chunk_path G:/ds-junior/final_work_ds/train_data\train_data_0.pq
chunk_path G:/ds-junior/final_work_ds/train_data\train_data_1.pq
['G:/ds-junior/final_work_ds/train_data\\train_data_0.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_1.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_10.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_11.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_2.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_3.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_4.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_5.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_6.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_7.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_8.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_9.pq', 'G:/ds-junior/final_work_ds/train_data\\train_target.csv']


Reading dataset with pandas:   0%|          | 0/2 [00:00<?, ?it/s]

...

chunk_path G:/ds-junior/final_work_ds/train_data\train_data_4.pq
chunk_path G:/ds-junior/final_work_ds/train_data\train_data_5.pq
['G:/ds-junior/final_work_ds/train_data\\train_data_0.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_1.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_10.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_11.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_2.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_3.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_4.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_5.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_6.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_7.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_8.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_9.pq', 'G:/ds-junior/final_work_ds/train_data\\train_target.csv']


Reading dataset with pandas:   0%|          | 0/2 [00:00<?, ?it/s]

chunk_path G:/ds-junior/final_work_ds/train_data\train_data_6.pq
chunk_path G:/ds-junior/final_work_ds/train_data\train_data_7.pq
['G:/ds-junior/final_work_ds/train_data\\train_data_0.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_1.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_10.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_11.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_2.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_3.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_4.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_5.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_6.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_7.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_8.pq', 'G:/ds-junior/final_work_ds/train_data\\train_data_9.pq', 'G:/ds-junior/final_work_ds/train_data\\train_target.csv']


Reading dataset with pandas:   0%|          | 0/2 [00:00<?, ?it/s]

chunk_path G:/ds-junior/final_work_ds/train_data\train_data_8.pq
chunk_path G:/ds-junior/final_work_ds/train_data\train_data_9.pq


In [6]:
#пример полученных данных
pd.set_option('display.max_columns', None)
data.head()

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,pre_loans_outstanding,pre_loans_total_overdue,pre_loans_max_overdue_sum,pre_loans_credit_cost_rate,pre_loans5,pre_loans530,pre_loans3060,pre_loans6090,pre_loans90,is_zero_loans5,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,pre_util,pre_over2limit,pre_maxover2limit,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,enc_paym_0,enc_paym_1,enc_paym_2,enc_paym_3,enc_paym_4,enc_paym_5,enc_paym_6,enc_paym_7,enc_paym_8,enc_paym_9,enc_paym_10,enc_paym_11,enc_paym_12,enc_paym_13,enc_paym_14,enc_paym_15,enc_paym_16,enc_paym_17,enc_paym_18,enc_paym_19,enc_paym_20,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag
0,0,1,18,9,2,3,16,10,11,3,3,0,2,11,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,3,3,3,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0
1,0,2,18,9,14,14,12,12,0,3,3,0,2,11,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,4,1,3,4,1,0,0
2,0,3,18,9,4,8,1,11,11,0,5,0,2,8,6,16,5,4,8,1,1,1,1,1,15,2,17,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,4,1,2,3,1,1,1
3,0,4,4,1,9,12,16,7,12,2,3,0,2,4,6,16,5,4,8,0,1,1,1,1,16,2,17,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,1,1,0,0
4,0,5,5,12,15,2,11,12,10,2,3,0,2,4,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0


# Объединение данных в датасет

In [7]:
# значения целевой переменной (смерджить с фичами по id)
targets = pd.read_csv('G:/ds-junior/final_work_ds/train_data/train_target.csv')
targets.head()

,id,flag
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [8]:
data.shape

(26162717, 61)

In [9]:
targets.shape

(3000000, 2)

In [10]:
df = data.merge(targets, on='id')

In [11]:
df.shape

(26162717, 62)

# Описание признаков датасета

* id - Идентификатор заявки. Заявки пронумерованы так, что большему номеру соответствует более поздняя дата заявки.
* rn - Порядковый номер кредитного продукта в кредитной истории. Большему номеру соответствует продукт с более поздней датой открытия.
* pre_since_opened - Дней с даты открытия кредита до даты сбора данных (бинаризовано*)
* pre_since_confirmed	- Дней с даты подтверждения информации по кредиту до даты сбора данных (бинаризовано*)
* pre_pterm - Плановое количество дней с даты открытия кредита до даты закрытия (бинаризовано*)
* pre_fterm - Фактическое количество дней с даты открытия кредита до даты закрытия (бинаризовано*)
* pre_till_pclose - Плановое количество дней с даты сбора данных до даты закрытия кредита (бинаризовано*)
* pre_till_fclose - Фактическое количество дней с даты сбора данных до даты закрытия кредита (бинаризовано*)
* pre_loans_credit_limit - Кредитный лимит (бинаризовано*)
* pre_loans_next_pay_summ - Сумма следующего платежа по кредиту (бинаризовано*)
* pre_loans_outstanding - Оставшаяся невыплаченная сумма кредита (бинаризовано*)
* pre_loans_total_overdue - Текущая просроченная задолженность (бинаризовано*)
* pre_loans_max_overdue_sum - Максимальная просроченная задолженность (бинаризовано*)
* pre_loans_credit_cost_rate - Полная стоимость кредита (бинаризовано*)
* pre_loans5 - Число просрочек до 5 дней (бинаризовано*)
* pre_loans530 - Число просрочек от 5 до 30 дней (бинаризовано*)
* pre_loans3060 - Число просрочек от 30 до 60 дней (бинаризовано*)
* pre_loans6090 - Число просрочек от 60 до 90 дней (бинаризовано*)
* pre_loans90	- Число просрочек более, чем на 90 дней (бинаризовано*)
* is_zero_loans_5	- Флаг: нет просрочек до 5 дней
* is_zero_loans_530 - Флаг: нет просрочек от 5 до 30 дней
* is_zero_loans_3060 - Флаг: нет просрочек от 30 до 60 дней
* is_zero_loans_6090 - Флаг: нет просрочек от 60 до 90 дней
* is_zero_loans90 - Флаг: нет просрочек более, чем на 90 дней
* pre_util - Отношение оставшейся невыплаченной суммы кредита к кредитному лимиту (бинаризовано*)
* pre_over2limit - Отношение текущей просроченной задолженности к кредитному лимиту (бинаризовано*)
* pre_maxover2limit - Отношенение максимальной просроченной задолженности к кредитному лимиту (бинаризовано*)
* is_zero_util - Флаг: отношение оставшейся невыплаченной суммы кредита к кредитному лимиту равняется 0
* is_zero_over2limit - Флаг: отношение текущей просроченной задолженности к кредитному лимиту равняется 0
* is_zero_maxover2limit - Флаг: отношение максимальной просроченной задолженности к кредитному лимиту равняется 0
* enc_paym_{0..N} - Статусы ежемесячных платежей за последние N месяцев (закодировано**)
* enc_loans_account_holder_type - Тип отношения к кредиту (закодировано**)
* enc_loans_credit_status	- Статус кредита (закодировано**)
* enc_loans_account_cur - Валюта кредита (закодировано**)
* enc_loans_credit_type - Тип кредита (закодировано**)
* pclose_flag - Флаг: плановое количество дней с даты открытия кредита до даты закрытия не определено
* fclose_flag - Флаг: фактическое количество дней с даты открытия кредита до даты закрытия не определено

* *область значений поля разбивается на N непересекающихся промежутков, каждому промежутку случайным образом ставится в соответствие уникальный номер от 0 до N-1, значение поля заменяется номером промежутка, которому оно принадлежит

* **каждому уникальному значению поля случайным образом ставится в соответствие уникальный номер от 0 до K, значение поля заменяется номером этого значения

# Информация о датасете

In [21]:
row_0 = df.iloc[0]
columns_with_first_values = [f'"{col}": {val},' for col, val in row_0.items()]
columns_with_first_values[-1] = columns_with_first_values[-1].rstrip(',')
for item in columns_with_first_values:
    print(item)

"id": 0,
"rn": 1,
"pre_since_opened": 18,
"pre_since_confirmed": 9,
"pre_pterm": 2,
"pre_fterm": 3,
"pre_till_pclose": 16,
"pre_till_fclose": 10,
"pre_loans_credit_limit": 11,
"pre_loans_next_pay_summ": 3,
"pre_loans_outstanding": 3,
"pre_loans_total_overdue": 0,
"pre_loans_max_overdue_sum": 2,
"pre_loans_credit_cost_rate": 11,
"pre_loans5": 6,
"pre_loans530": 16,
"pre_loans3060": 5,
"pre_loans6090": 4,
"pre_loans90": 8,
"is_zero_loans5": 1,
"is_zero_loans530": 1,
"is_zero_loans3060": 1,
"is_zero_loans6090": 1,
"is_zero_loans90": 1,
"pre_util": 16,
"pre_over2limit": 2,
"pre_maxover2limit": 17,
"is_zero_util": 1,
"is_zero_over2limit": 1,
"is_zero_maxover2limit": 1,
"enc_paym_0": 0,
"enc_paym_1": 0,
"enc_paym_2": 3,
"enc_paym_3": 3,
"enc_paym_4": 3,
"enc_paym_5": 3,
"enc_paym_6": 3,
"enc_paym_7": 3,
"enc_paym_8": 3,
"enc_paym_9": 3,
"enc_paym_10": 3,
"enc_paym_11": 4,
"enc_paym_12": 3,
"enc_paym_13": 3,
"enc_paym_14": 3,
"enc_paym_15": 3,
"enc_paym_16": 3,
"enc_paym_17": 3,
"enc_paym_18"

In [22]:
row_1 = df.iloc[1]
columns_with_first_values = [f'"{col}": {val},' for col, val in row_1.items()]
columns_with_first_values[-1] = columns_with_first_values[-1].rstrip(',')
for item in columns_with_first_values:
    print(item)

"id": 0,
"rn": 2,
"pre_since_opened": 18,
"pre_since_confirmed": 9,
"pre_pterm": 14,
"pre_fterm": 14,
"pre_till_pclose": 12,
"pre_till_fclose": 12,
"pre_loans_credit_limit": 0,
"pre_loans_next_pay_summ": 3,
"pre_loans_outstanding": 3,
"pre_loans_total_overdue": 0,
"pre_loans_max_overdue_sum": 2,
"pre_loans_credit_cost_rate": 11,
"pre_loans5": 6,
"pre_loans530": 16,
"pre_loans3060": 5,
"pre_loans6090": 4,
"pre_loans90": 8,
"is_zero_loans5": 1,
"is_zero_loans530": 1,
"is_zero_loans3060": 1,
"is_zero_loans6090": 1,
"is_zero_loans90": 1,
"pre_util": 16,
"pre_over2limit": 2,
"pre_maxover2limit": 17,
"is_zero_util": 1,
"is_zero_over2limit": 1,
"is_zero_maxover2limit": 1,
"enc_paym_0": 0,
"enc_paym_1": 0,
"enc_paym_2": 0,
"enc_paym_3": 0,
"enc_paym_4": 0,
"enc_paym_5": 0,
"enc_paym_6": 0,
"enc_paym_7": 0,
"enc_paym_8": 0,
"enc_paym_9": 0,
"enc_paym_10": 0,
"enc_paym_11": 1,
"enc_paym_12": 0,
"enc_paym_13": 0,
"enc_paym_14": 0,
"enc_paym_15": 0,
"enc_paym_16": 0,
"enc_paym_17": 0,
"enc_paym_18

In [23]:
row_2 = df.iloc[2]
columns_with_first_values = [f'"{col}": {val},' for col, val in row_2.items()]
columns_with_first_values[-1] = columns_with_first_values[-1].rstrip(',')
for item in columns_with_first_values:
    print(item)

"id": 0,
"rn": 3,
"pre_since_opened": 18,
"pre_since_confirmed": 9,
"pre_pterm": 4,
"pre_fterm": 8,
"pre_till_pclose": 1,
"pre_till_fclose": 11,
"pre_loans_credit_limit": 11,
"pre_loans_next_pay_summ": 0,
"pre_loans_outstanding": 5,
"pre_loans_total_overdue": 0,
"pre_loans_max_overdue_sum": 2,
"pre_loans_credit_cost_rate": 8,
"pre_loans5": 6,
"pre_loans530": 16,
"pre_loans3060": 5,
"pre_loans6090": 4,
"pre_loans90": 8,
"is_zero_loans5": 1,
"is_zero_loans530": 1,
"is_zero_loans3060": 1,
"is_zero_loans6090": 1,
"is_zero_loans90": 1,
"pre_util": 15,
"pre_over2limit": 2,
"pre_maxover2limit": 17,
"is_zero_util": 0,
"is_zero_over2limit": 1,
"is_zero_maxover2limit": 1,
"enc_paym_0": 0,
"enc_paym_1": 0,
"enc_paym_2": 0,
"enc_paym_3": 0,
"enc_paym_4": 0,
"enc_paym_5": 0,
"enc_paym_6": 0,
"enc_paym_7": 0,
"enc_paym_8": 0,
"enc_paym_9": 0,
"enc_paym_10": 0,
"enc_paym_11": 1,
"enc_paym_12": 0,
"enc_paym_13": 0,
"enc_paym_14": 0,
"enc_paym_15": 0,
"enc_paym_16": 0,
"enc_paym_17": 0,
"enc_paym_18": 

In [24]:
row_3 = df.iloc[3]
columns_with_first_values = [f'"{col}": {val},' for col, val in row_3.items()]
columns_with_first_values[-1] = columns_with_first_values[-1].rstrip(',')
for item in columns_with_first_values:
    print(item)

"id": 0,
"rn": 4,
"pre_since_opened": 4,
"pre_since_confirmed": 1,
"pre_pterm": 9,
"pre_fterm": 12,
"pre_till_pclose": 16,
"pre_till_fclose": 7,
"pre_loans_credit_limit": 12,
"pre_loans_next_pay_summ": 2,
"pre_loans_outstanding": 3,
"pre_loans_total_overdue": 0,
"pre_loans_max_overdue_sum": 2,
"pre_loans_credit_cost_rate": 4,
"pre_loans5": 6,
"pre_loans530": 16,
"pre_loans3060": 5,
"pre_loans6090": 4,
"pre_loans90": 8,
"is_zero_loans5": 0,
"is_zero_loans530": 1,
"is_zero_loans3060": 1,
"is_zero_loans6090": 1,
"is_zero_loans90": 1,
"pre_util": 16,
"pre_over2limit": 2,
"pre_maxover2limit": 17,
"is_zero_util": 1,
"is_zero_over2limit": 1,
"is_zero_maxover2limit": 1,
"enc_paym_0": 1,
"enc_paym_1": 0,
"enc_paym_2": 0,
"enc_paym_3": 0,
"enc_paym_4": 0,
"enc_paym_5": 0,
"enc_paym_6": 0,
"enc_paym_7": 0,
"enc_paym_8": 0,
"enc_paym_9": 0,
"enc_paym_10": 0,
"enc_paym_11": 1,
"enc_paym_12": 3,
"enc_paym_13": 3,
"enc_paym_14": 3,
"enc_paym_15": 3,
"enc_paym_16": 3,
"enc_paym_17": 3,
"enc_paym_18": 

In [25]:
row_4 = df.iloc[4]
columns_with_first_values = [f'"{col}": {val},' for col, val in row_4.items()]
columns_with_first_values[-1] = columns_with_first_values[-1].rstrip(',')
for item in columns_with_first_values:
    print(item)

"id": 0,
"rn": 5,
"pre_since_opened": 5,
"pre_since_confirmed": 12,
"pre_pterm": 15,
"pre_fterm": 2,
"pre_till_pclose": 11,
"pre_till_fclose": 12,
"pre_loans_credit_limit": 10,
"pre_loans_next_pay_summ": 2,
"pre_loans_outstanding": 3,
"pre_loans_total_overdue": 0,
"pre_loans_max_overdue_sum": 2,
"pre_loans_credit_cost_rate": 4,
"pre_loans5": 6,
"pre_loans530": 16,
"pre_loans3060": 5,
"pre_loans6090": 4,
"pre_loans90": 8,
"is_zero_loans5": 1,
"is_zero_loans530": 1,
"is_zero_loans3060": 1,
"is_zero_loans6090": 1,
"is_zero_loans90": 1,
"pre_util": 16,
"pre_over2limit": 2,
"pre_maxover2limit": 17,
"is_zero_util": 1,
"is_zero_over2limit": 1,
"is_zero_maxover2limit": 1,
"enc_paym_0": 0,
"enc_paym_1": 0,
"enc_paym_2": 0,
"enc_paym_3": 0,
"enc_paym_4": 0,
"enc_paym_5": 0,
"enc_paym_6": 0,
"enc_paym_7": 3,
"enc_paym_8": 3,
"enc_paym_9": 3,
"enc_paym_10": 3,
"enc_paym_11": 4,
"enc_paym_12": 3,
"enc_paym_13": 3,
"enc_paym_14": 3,
"enc_paym_15": 3,
"enc_paym_16": 3,
"enc_paym_17": 3,
"enc_paym_18"

In [26]:
row_5 = df.iloc[5]
columns_with_first_values = [f'"{col}": {val},' for col, val in row_5.items()]
columns_with_first_values[-1] = columns_with_first_values[-1].rstrip(',')
for item in columns_with_first_values:
    print(item)

"id": 0,
"rn": 6,
"pre_since_opened": 5,
"pre_since_confirmed": 0,
"pre_pterm": 11,
"pre_fterm": 8,
"pre_till_pclose": 12,
"pre_till_fclose": 11,
"pre_loans_credit_limit": 4,
"pre_loans_next_pay_summ": 2,
"pre_loans_outstanding": 3,
"pre_loans_total_overdue": 0,
"pre_loans_max_overdue_sum": 2,
"pre_loans_credit_cost_rate": 4,
"pre_loans5": 6,
"pre_loans530": 16,
"pre_loans3060": 5,
"pre_loans6090": 4,
"pre_loans90": 8,
"is_zero_loans5": 1,
"is_zero_loans530": 1,
"is_zero_loans3060": 1,
"is_zero_loans6090": 1,
"is_zero_loans90": 1,
"pre_util": 9,
"pre_over2limit": 5,
"pre_maxover2limit": 4,
"is_zero_util": 0,
"is_zero_over2limit": 0,
"is_zero_maxover2limit": 0,
"enc_paym_0": 0,
"enc_paym_1": 0,
"enc_paym_2": 0,
"enc_paym_3": 0,
"enc_paym_4": 0,
"enc_paym_5": 0,
"enc_paym_6": 0,
"enc_paym_7": 0,
"enc_paym_8": 0,
"enc_paym_9": 0,
"enc_paym_10": 0,
"enc_paym_11": 1,
"enc_paym_12": 0,
"enc_paym_13": 0,
"enc_paym_14": 0,
"enc_paym_15": 0,
"enc_paym_16": 0,
"enc_paym_17": 0,
"enc_paym_18": 0,

In [27]:
row_6 = df.iloc[6]
columns_with_first_values = [f'"{col}": {val},' for col, val in row_6.items()]
columns_with_first_values[-1] = columns_with_first_values[-1].rstrip(',')
for item in columns_with_first_values:
    print(item)

"id": 0,
"rn": 7,
"pre_since_opened": 3,
"pre_since_confirmed": 9,
"pre_pterm": 1,
"pre_fterm": 2,
"pre_till_pclose": 12,
"pre_till_fclose": 14,
"pre_loans_credit_limit": 15,
"pre_loans_next_pay_summ": 5,
"pre_loans_outstanding": 3,
"pre_loans_total_overdue": 0,
"pre_loans_max_overdue_sum": 2,
"pre_loans_credit_cost_rate": 3,
"pre_loans5": 6,
"pre_loans530": 16,
"pre_loans3060": 5,
"pre_loans6090": 4,
"pre_loans90": 8,
"is_zero_loans5": 1,
"is_zero_loans530": 1,
"is_zero_loans3060": 1,
"is_zero_loans6090": 1,
"is_zero_loans90": 1,
"pre_util": 16,
"pre_over2limit": 2,
"pre_maxover2limit": 17,
"is_zero_util": 1,
"is_zero_over2limit": 1,
"is_zero_maxover2limit": 1,
"enc_paym_0": 0,
"enc_paym_1": 0,
"enc_paym_2": 0,
"enc_paym_3": 0,
"enc_paym_4": 0,
"enc_paym_5": 0,
"enc_paym_6": 0,
"enc_paym_7": 0,
"enc_paym_8": 3,
"enc_paym_9": 3,
"enc_paym_10": 3,
"enc_paym_11": 4,
"enc_paym_12": 3,
"enc_paym_13": 3,
"enc_paym_14": 3,
"enc_paym_15": 3,
"enc_paym_16": 3,
"enc_paym_17": 3,
"enc_paym_18": 

In [28]:
row_7 = df.iloc[7]
columns_with_first_values = [f'"{col}": {val},' for col, val in row_7.items()]
columns_with_first_values[-1] = columns_with_first_values[-1].rstrip(',')
for item in columns_with_first_values:
    print(item)

"id": 0,
"rn": 8,
"pre_since_opened": 2,
"pre_since_confirmed": 9,
"pre_pterm": 2,
"pre_fterm": 3,
"pre_till_pclose": 12,
"pre_till_fclose": 14,
"pre_loans_credit_limit": 15,
"pre_loans_next_pay_summ": 5,
"pre_loans_outstanding": 3,
"pre_loans_total_overdue": 0,
"pre_loans_max_overdue_sum": 2,
"pre_loans_credit_cost_rate": 13,
"pre_loans5": 6,
"pre_loans530": 16,
"pre_loans3060": 5,
"pre_loans6090": 4,
"pre_loans90": 8,
"is_zero_loans5": 1,
"is_zero_loans530": 1,
"is_zero_loans3060": 1,
"is_zero_loans6090": 1,
"is_zero_loans90": 1,
"pre_util": 16,
"pre_over2limit": 2,
"pre_maxover2limit": 17,
"is_zero_util": 1,
"is_zero_over2limit": 1,
"is_zero_maxover2limit": 1,
"enc_paym_0": 0,
"enc_paym_1": 0,
"enc_paym_2": 3,
"enc_paym_3": 3,
"enc_paym_4": 3,
"enc_paym_5": 3,
"enc_paym_6": 3,
"enc_paym_7": 3,
"enc_paym_8": 3,
"enc_paym_9": 3,
"enc_paym_10": 3,
"enc_paym_11": 4,
"enc_paym_12": 3,
"enc_paym_13": 3,
"enc_paym_14": 3,
"enc_paym_15": 3,
"enc_paym_16": 3,
"enc_paym_17": 3,
"enc_paym_18":

In [29]:
row_8 = df.iloc[8]
columns_with_first_values = [f'"{col}": {val},' for col, val in row_8.items()]
columns_with_first_values[-1] = columns_with_first_values[-1].rstrip(',')
for item in columns_with_first_values:
    print(item)

"id": 0,
"rn": 9,
"pre_since_opened": 1,
"pre_since_confirmed": 9,
"pre_pterm": 11,
"pre_fterm": 13,
"pre_till_pclose": 14,
"pre_till_fclose": 8,
"pre_loans_credit_limit": 2,
"pre_loans_next_pay_summ": 5,
"pre_loans_outstanding": 1,
"pre_loans_total_overdue": 0,
"pre_loans_max_overdue_sum": 2,
"pre_loans_credit_cost_rate": 11,
"pre_loans5": 6,
"pre_loans530": 16,
"pre_loans3060": 5,
"pre_loans6090": 4,
"pre_loans90": 8,
"is_zero_loans5": 1,
"is_zero_loans530": 1,
"is_zero_loans3060": 1,
"is_zero_loans6090": 1,
"is_zero_loans90": 1,
"pre_util": 1,
"pre_over2limit": 2,
"pre_maxover2limit": 17,
"is_zero_util": 0,
"is_zero_over2limit": 1,
"is_zero_maxover2limit": 1,
"enc_paym_0": 0,
"enc_paym_1": 0,
"enc_paym_2": 0,
"enc_paym_3": 0,
"enc_paym_4": 0,
"enc_paym_5": 0,
"enc_paym_6": 0,
"enc_paym_7": 0,
"enc_paym_8": 0,
"enc_paym_9": 0,
"enc_paym_10": 0,
"enc_paym_11": 1,
"enc_paym_12": 0,
"enc_paym_13": 0,
"enc_paym_14": 3,
"enc_paym_15": 3,
"enc_paym_16": 3,
"enc_paym_17": 3,
"enc_paym_18": 

In [30]:
row_9 = df.iloc[9]
columns_with_first_values = [f'"{col}": {val},' for col, val in row_9.items()]
columns_with_first_values[-1] = columns_with_first_values[-1].rstrip(',')
for item in columns_with_first_values:
    print(item)

"id": 0,
"rn": 10,
"pre_since_opened": 7,
"pre_since_confirmed": 9,
"pre_pterm": 2,
"pre_fterm": 10,
"pre_till_pclose": 8,
"pre_till_fclose": 8,
"pre_loans_credit_limit": 16,
"pre_loans_next_pay_summ": 4,
"pre_loans_outstanding": 2,
"pre_loans_total_overdue": 0,
"pre_loans_max_overdue_sum": 2,
"pre_loans_credit_cost_rate": 11,
"pre_loans5": 6,
"pre_loans530": 16,
"pre_loans3060": 5,
"pre_loans6090": 4,
"pre_loans90": 8,
"is_zero_loans5": 1,
"is_zero_loans530": 1,
"is_zero_loans3060": 1,
"is_zero_loans6090": 1,
"is_zero_loans90": 1,
"pre_util": 15,
"pre_over2limit": 2,
"pre_maxover2limit": 17,
"is_zero_util": 0,
"is_zero_over2limit": 1,
"is_zero_maxover2limit": 1,
"enc_paym_0": 0,
"enc_paym_1": 0,
"enc_paym_2": 0,
"enc_paym_3": 0,
"enc_paym_4": 0,
"enc_paym_5": 0,
"enc_paym_6": 3,
"enc_paym_7": 3,
"enc_paym_8": 3,
"enc_paym_9": 3,
"enc_paym_10": 3,
"enc_paym_11": 4,
"enc_paym_12": 3,
"enc_paym_13": 3,
"enc_paym_14": 3,
"enc_paym_15": 3,
"enc_paym_16": 3,
"enc_paym_17": 3,
"enc_paym_18":

In [31]:
row_10 = df.iloc[10]
columns_with_first_values = [f'"{col}": {val},' for col, val in row_10.items()]
columns_with_first_values[-1] = columns_with_first_values[-1].rstrip(',')
for item in columns_with_first_values:
    print(item)

"id": 1,
"rn": 1,
"pre_since_opened": 8,
"pre_since_confirmed": 7,
"pre_pterm": 12,
"pre_fterm": 15,
"pre_till_pclose": 9,
"pre_till_fclose": 1,
"pre_loans_credit_limit": 1,
"pre_loans_next_pay_summ": 2,
"pre_loans_outstanding": 4,
"pre_loans_total_overdue": 0,
"pre_loans_max_overdue_sum": 1,
"pre_loans_credit_cost_rate": 4,
"pre_loans5": 6,
"pre_loans530": 16,
"pre_loans3060": 5,
"pre_loans6090": 4,
"pre_loans90": 8,
"is_zero_loans5": 1,
"is_zero_loans530": 1,
"is_zero_loans3060": 0,
"is_zero_loans6090": 0,
"is_zero_loans90": 0,
"pre_util": 2,
"pre_over2limit": 10,
"pre_maxover2limit": 8,
"is_zero_util": 0,
"is_zero_over2limit": 0,
"is_zero_maxover2limit": 0,
"enc_paym_0": 0,
"enc_paym_1": 3,
"enc_paym_2": 3,
"enc_paym_3": 3,
"enc_paym_4": 3,
"enc_paym_5": 3,
"enc_paym_6": 3,
"enc_paym_7": 3,
"enc_paym_8": 3,
"enc_paym_9": 2,
"enc_paym_10": 0,
"enc_paym_11": 1,
"enc_paym_12": 0,
"enc_paym_13": 0,
"enc_paym_14": 0,
"enc_paym_15": 0,
"enc_paym_16": 0,
"enc_paym_17": 0,
"enc_paym_18": 0,

In [32]:
row_11 = df.iloc[11]
columns_with_first_values = [f'"{col}": {val},' for col, val in row_11.items()]
columns_with_first_values[-1] = columns_with_first_values[-1].rstrip(',')
for item in columns_with_first_values:
    print(item)

"id": 1,
"rn": 2,
"pre_since_opened": 8,
"pre_since_confirmed": 7,
"pre_pterm": 13,
"pre_fterm": 2,
"pre_till_pclose": 9,
"pre_till_fclose": 1,
"pre_loans_credit_limit": 6,
"pre_loans_next_pay_summ": 2,
"pre_loans_outstanding": 3,
"pre_loans_total_overdue": 0,
"pre_loans_max_overdue_sum": 1,
"pre_loans_credit_cost_rate": 4,
"pre_loans5": 6,
"pre_loans530": 16,
"pre_loans3060": 5,
"pre_loans6090": 4,
"pre_loans90": 8,
"is_zero_loans5": 1,
"is_zero_loans530": 1,
"is_zero_loans3060": 0,
"is_zero_loans6090": 0,
"is_zero_loans90": 0,
"pre_util": 0,
"pre_over2limit": 0,
"pre_maxover2limit": 10,
"is_zero_util": 0,
"is_zero_over2limit": 0,
"is_zero_maxover2limit": 0,
"enc_paym_0": 0,
"enc_paym_1": 3,
"enc_paym_2": 3,
"enc_paym_3": 3,
"enc_paym_4": 3,
"enc_paym_5": 3,
"enc_paym_6": 3,
"enc_paym_7": 2,
"enc_paym_8": 0,
"enc_paym_9": 0,
"enc_paym_10": 0,
"enc_paym_11": 1,
"enc_paym_12": 0,
"enc_paym_13": 3,
"enc_paym_14": 3,
"enc_paym_15": 3,
"enc_paym_16": 3,
"enc_paym_17": 3,
"enc_paym_18": 3,


In [33]:
row_12 = df.iloc[12]
columns_with_first_values = [f'"{col}": {val},' for col, val in row_12.items()]
columns_with_first_values[-1] = columns_with_first_values[-1].rstrip(',')
for item in columns_with_first_values:
    print(item)

"id": 1,
"rn": 3,
"pre_since_opened": 15,
"pre_since_confirmed": 7,
"pre_pterm": 7,
"pre_fterm": 6,
"pre_till_pclose": 9,
"pre_till_fclose": 1,
"pre_loans_credit_limit": 6,
"pre_loans_next_pay_summ": 2,
"pre_loans_outstanding": 3,
"pre_loans_total_overdue": 0,
"pre_loans_max_overdue_sum": 2,
"pre_loans_credit_cost_rate": 4,
"pre_loans5": 6,
"pre_loans530": 16,
"pre_loans3060": 5,
"pre_loans6090": 4,
"pre_loans90": 8,
"is_zero_loans5": 1,
"is_zero_loans530": 1,
"is_zero_loans3060": 1,
"is_zero_loans6090": 1,
"is_zero_loans90": 1,
"pre_util": 16,
"pre_over2limit": 2,
"pre_maxover2limit": 17,
"is_zero_util": 1,
"is_zero_over2limit": 1,
"is_zero_maxover2limit": 1,
"enc_paym_0": 0,
"enc_paym_1": 0,
"enc_paym_2": 0,
"enc_paym_3": 0,
"enc_paym_4": 0,
"enc_paym_5": 3,
"enc_paym_6": 3,
"enc_paym_7": 3,
"enc_paym_8": 3,
"enc_paym_9": 3,
"enc_paym_10": 3,
"enc_paym_11": 4,
"enc_paym_12": 3,
"enc_paym_13": 3,
"enc_paym_14": 3,
"enc_paym_15": 3,
"enc_paym_16": 3,
"enc_paym_17": 3,
"enc_paym_18": 3,

In [19]:
def convert_dtype(dtype):
    if dtype in ['int32', 'int64', 'int16', 'int8']:
        return 'int'
    elif dtype in ['float32', 'float64']:
        return 'float'
    return dtype

# Создаем список столбцов с типами данных
columns_with_types = [f"{col}: {convert_dtype(dtype.name)}" for col, dtype in df.dtypes.items()]

# Выводим список без кавычек
for item in columns_with_types:
    print(item)

id: int
rn: int
pre_since_opened: int
pre_since_confirmed: int
pre_pterm: int
pre_fterm: int
pre_till_pclose: int
pre_till_fclose: int
pre_loans_credit_limit: int
pre_loans_next_pay_summ: int
pre_loans_outstanding: int
pre_loans_total_overdue: int
pre_loans_max_overdue_sum: int
pre_loans_credit_cost_rate: int
pre_loans5: int
pre_loans530: int
pre_loans3060: int
pre_loans6090: int
pre_loans90: int
is_zero_loans5: int
is_zero_loans530: int
is_zero_loans3060: int
is_zero_loans6090: int
is_zero_loans90: int
pre_util: int
pre_over2limit: int
pre_maxover2limit: int
is_zero_util: int
is_zero_over2limit: int
is_zero_maxover2limit: int
enc_paym_0: int
enc_paym_1: int
enc_paym_2: int
enc_paym_3: int
enc_paym_4: int
enc_paym_5: int
enc_paym_6: int
enc_paym_7: int
enc_paym_8: int
enc_paym_9: int
enc_paym_10: int
enc_paym_11: int
enc_paym_12: int
enc_paym_13: int
enc_paym_14: int
enc_paym_15: int
enc_paym_16: int
enc_paym_17: int
enc_paym_18: int
enc_paym_19: int
enc_paym_20: int
enc_paym_21: int
e

In [131]:
# Посмотрим какая информация содержится в датасете
df.info()

In [132]:
# Проверим данные на пропуски
df.isna().sum()

In [133]:
# Пропусков нет

In [134]:
# Посмотрим описание датасета
# df.describe().round(2)

In [135]:
# Создаем функцию для подсчета уникальных значений в каждом признаке
def print_unique_count(df):
    for column in df.columns:
        unique_count = df[column].nunique()
        print(f"Признак: {column}")
        print(f"Количество уникальных значений: {unique_count}\n")

In [136]:
# Посмотрим количество уникальных значений в каждом признаке датасета
print_unique_count(df)

# Визуализация значений признаков

In [137]:
# pre_since_opened - Дней с даты открытия кредита до даты сбора данных (бинаризовано*)
sns.countplot(x='pre_since_opened', data=df);

In [138]:
# pre_since_confirmed	- Дней с даты подтверждения информации по кредиту до даты сбора данных (бинаризовано*)
sns.countplot(x='pre_since_confirmed', data=df);

In [139]:
# pre_pterm - Плановое количество дней с даты открытия кредита до даты закрытия (бинаризовано*)
sns.countplot(x='pre_pterm', data=df);

In [140]:
# pre_fterm - Фактическое количество дней с даты открытия кредита до даты закрытия (бинаризовано*) \
sns.countplot(x='pre_fterm', data=df);

In [141]:
# pre_till_pclose - Плановое количество дней с даты сбора данных до даты закрытия кредита (бинаризовано*)
sns.countplot(x='pre_till_pclose', data=df);

In [142]:
# pre_till_fclose - Фактическое количество дней с даты сбора данных до даты закрытия кредита (бинаризовано*)
sns.countplot(x='pre_till_fclose', data=df);

In [143]:
# pre_loans_credit_limit - Кредитный лимит (бинаризовано*)
sns.countplot(x='pre_loans_credit_limit', data=df);

In [144]:
# pre_loans_next_pay_summ - Сумма следующего платежа по кредиту (бинаризовано*)
sns.countplot(x='pre_loans_next_pay_summ', data=df);

In [145]:
# pre_loans_outstanding - Оставшаяся невыплаченная сумма кредита (бинаризовано*)
sns.countplot(x='pre_loans_outstanding', data=df);

In [146]:
# pre_loans_total_overdue - Текущая просроченная задолженность (бинаризовано*)
sns.countplot(x='pre_loans_total_overdue', data=df);

In [147]:
# pre_loans_max_overdue_sum - Максимальная просроченная задолженность (бинаризовано*)
sns.countplot(x='pre_loans_max_overdue_sum', data=df);

In [148]:
# pre_loans_credit_cost_rate - Полная стоимость кредита (бинаризовано*)
sns.countplot(x='pre_loans_credit_cost_rate', data=df);

In [149]:
# pre_loans5 - Число просрочек до 5 дней (бинаризовано*)
sns.countplot(x='pre_loans5', data=df);

In [150]:
# pre_loans530 - Число просрочек от 5 до 30 дней (бинаризовано*)
sns.countplot(x='pre_loans530', data=df);

In [151]:
# pre_loans6090 - Число просрочек от 60 до 90 дней (бинаризовано*)
sns.countplot(x='pre_loans6090', data=df);

In [152]:
# pre_loans90	- Число просрочек более, чем на 90 дней (бинаризовано*)
sns.countplot(x='pre_loans90', data=df);

In [153]:
# pre_util - Отношение оставшейся невыплаченной суммы кредита к кредитному лимиту (бинаризовано*)
sns.countplot(x='pre_util', data=df);

In [154]:
# pre_over2limit - Отношение текущей просроченной задолженности к кредитному лимиту (бинаризовано*)
sns.countplot(x='pre_over2limit', data=df);

In [155]:
# is_zero_util - Флаг: отношение оставшейся невыплаченной суммы кредита к кредитному лимиту равняется 0
sns.countplot(x='is_zero_util', data=df);

In [156]:
# is_zero_over2limit - Флаг: отношение текущей просроченной задолженности к кредитному лимиту равняется 0
sns.countplot(x='is_zero_over2limit', data=df);

In [157]:
# is_zero_maxover2limit - Флаг: отношение максимальной просроченной задолженности к кредитному лимиту равняется 0
sns.countplot(x='is_zero_maxover2limit', data=df);

In [158]:
# enc_paym_{0..N} - Статусы ежемесячных платежей за последние N месяцев (закодировано**)
sns.countplot(x='enc_paym_1', data=df);

In [159]:
# enc_loans_account_holder_type - Тип отношения к кредиту (закодировано**)
sns.countplot(x='enc_loans_account_holder_type', data=df);

In [160]:
# enc_loans_credit_status	- Статус кредита (закодировано**)
sns.countplot(x='enc_loans_credit_status', data=df);

In [161]:
# enc_loans_account_cur - Валюта кредита (закодировано**)
sns.countplot(x='enc_loans_account_cur', data=df);

In [162]:
# enc_loans_credit_type - Тип кредита (закодировано**)
sns.countplot(x='enc_loans_credit_type', data=df);

In [163]:
# pclose_flag - Флаг: плановое количество дней с даты открытия кредита до даты закрытия не определено
sns.countplot(x='pclose_flag', data=df);

In [164]:
# fclose_flag - Флаг: фактическое количество дней с даты открытия кредита до даты закрытия не определено
sns.countplot(x='fclose_flag', data=df);

# Создание новых признаков

In [165]:
# Cумма флагов, указывающих на отсутствие просрочек
df['total_overdue_flags'] = (
        df['is_zero_loans5'] +
        df['is_zero_loans530'] +
        df['is_zero_loans3060'] +
        df['is_zero_loans6090'] +
        df['is_zero_loans90']
)

In [166]:
# Общее количество просрочек
df['total_overdue_counts'] = (
        df['pre_loans5'] +
        df['pre_loans530'] +
        df['pre_loans3060'] +
        df['pre_loans6090'] +
        df['pre_loans90']
)

In [167]:
# Длинна кредитной истории
df['credit_history_len'] = (
    df['pre_since_opened'] - 
    df['pre_since_confirmed']
)

In [168]:
# Разница между плановой и фактической датой открытия кредита
df['close_flags'] = (
    df['pclose_flag'] - 
    df['fclose_flag']
)

# Снижение размерности датасета 1

In [169]:
# Снизим размерность признаков датасета для уменьшения объема
col = df.columns
df[col] = df[col].apply(pd.to_numeric, downcast="integer")

In [170]:
#Нам удалось снизить размер датасета с 12 GB до 1.7 GB
df.info()

In [171]:
# Посмотрим описательные характеристики датасета
df.describe().round()

# Объединение признаков по id

In [172]:
# Создадим копию DataFrame, содержащая только столбцы 'id', 'rn' и 'flag'. 
prepared_df = df[['id', 'rn', 'flag']].copy()

In [173]:
# Найдем все столбцы в исходном DataFrame, кроме 'id', 'rn' и 'flag', и сохраняет их в списке features.
features = list(set(df.columns) - set(['id', 'rn', 'flag']))

In [174]:
# Здесь применяется функция pd.get_dummies для создания dummy (бинарных) признаков для всех категориальных переменных, указанных в features, путем одномерного кодирования
dummies = pd.get_dummies(df[features], columns=features)

In [175]:
# dummy-признаки добавляются к ранее подготовленному DataFrame, состоящему из столбцов 'id', 'rn' и 'flag'.
prepared_df = pd.concat([prepared_df, dummies], axis=1)

In [176]:
# Здесь создается словарь agg_df, в котором ключами являются имена признаков, для которых были созданы dummy-признаки (то есть все признаки, кроме 'id', 'rn' и 'flag'), а значениями — строка 'sum', которая указывает, что данные по этим признакам будут агрегированы с помощью суммы.
agg_df = {f: 'sum' for f in set(prepared_df.columns) - set(['id', 'rn', 'flag'])}

In [177]:
# Устанавливается, что для столбца 'rn' будет применена агрегирующая функция 'count', которая подсчитывает количество записей.
agg_df['rn'] = 'count'

In [178]:
# Выполняется группировка DataFrame по значениям столбцов 'id' и 'flag', к каждой группе применяется агрегирующая функция, заданная в agg_df.
# Затем результат преобразуется в целочисленный тип с помощью .astype('int').
# И наконец, индекс сбрасывается с помощью reset_index(drop=False), чтобы столбцы 'id' и 'flag' снова стали частью данных.
prepared_df = prepared_df.groupby(['id', 'flag']).agg(agg_df).astype('int').reset_index(drop=False)

In [179]:
prepared_df.head()

In [180]:
agg_df = prepared_df

# Удаление неинформативных признаков и признаков с одним значением

In [181]:
# Определим признаки с одним значением
single_valued_columns = agg_df.columns[agg_df.nunique() == 1]

In [182]:
# Удалим признаки с одним значением из датасета
agg_df = agg_df.drop(single_valued_columns, axis=1)

In [183]:
agg_df.shape

# Масштабирование признаков

In [184]:
# Здесь создается список binary_features, который содержит все столбцы из agg_df, имеющие ровно два уникальных значения. Это обычно признаки, которые принимают значения 0 и 1 (бинарные).
binary_features = [col for col in agg_df.columns if agg_df[col].nunique() == 2]

In [185]:
# Создается список continuous_features, который включает все столбцы из agg_df, кроме бинарных признаков, а также столбцов 'id' и 'flag'.
continuous_features = [col for col in agg_df.columns if col not in binary_features and col not in ['flag', 'id']]

In [186]:
scaler = StandardScaler()

In [187]:
agg_df[continuous_features] = scaler.fit_transform(agg_df[continuous_features])

In [188]:
# Удаляем столбец id, если он присутствует
if 'id' in agg_df.columns:
    agg_df.drop('id', axis=1, inplace=True)

In [189]:
# Проверяем на дублирование столбца flag 
if 'flag' not in agg_df.columns:
    agg_df['flag'] = flag

In [190]:
# Итоговый DataFrame
scaled_df = agg_df.copy()

In [191]:
scaled_df.head()

In [192]:
scaled_df.info()

# Снижение размерности датасета 2

In [193]:
# Приведение к целочисленным типам минимального размера:
scaled_df = scaled_df.apply(pd.to_numeric, downcast='integer')

In [194]:
# Приведение к вещественным типам минимального размера
scaled_df = scaled_df.apply(pd.to_numeric, downcast='float')

In [195]:
scaled_df.info()

# Разделение на тестовую и тренировочную выборки

In [196]:
X = scaled_df.drop(columns=['flag'], axis=1)
y = scaled_df['flag']

In [197]:
X.columns = X.columns.str.replace("-", "x")

In [198]:
X.info()

In [199]:
y.info()

In [200]:
# Разделение данных на обучающую и тестовую выборки
train_x, test_x, train_y, test_y = train_test_split(X, y, train_size = 0.8, random_state=42, stratify=y)

# Моделирование

In [201]:
# Рассчитываем веса образцов
sample_weights = compute_sample_weight(class_weight='balanced', y=train_y)

In [202]:
xgbc = XGBClassifier(learning_rate=0.2, n_estimators=400, max_depth=3)

In [203]:
# Обучаем XGBoost классификатор
xgbc.fit(train_x, train_y, sample_weight=sample_weights)

In [204]:
# Применение лучших параметров и оценка модели
y_pred = xgbc.predict(test_x)
y_pred_proba = xgbc.predict_proba(test_x)[:, 1]
print(f"AUC-ROC: {roc_auc_score(test_y, y_pred_proba)}")
print(classification_report(test_y, y_pred))

In [205]:
# 1. Высокая AUC-ROC указывает на то, что модель хорошо умеет различать классы в целом.

In [206]:
# 2. Precision для высокого риска очень низкая (0.07), что означает, что модель часто ошибается при предсказании высокорисковых заемщиков (много false positives).

In [207]:
# 3. Recall для высокого риска довольно высокое (0.70), что означает, что модель находит большинство высокорисковых заемщиков (немного ошибается, не предсказывая их).
# Думаю это наиболее важный показатель.

In [208]:
# 4. Для низкорисковых заемщиков модель показывает очень хорошие precision (0.98), что означает, что почти все предсказания низкого риска являются правильными, но средний recall (0.68) указывает, что многие низкорисковые заемщики были упущены.

# Порог предсказания вероятности

In [209]:
# Предсказание вероятностей для каждого класса
y_proba = xgbc.predict_proba(test_x)
y_true = test_y

In [210]:
# Функция для получения метрик на разных порогах
def evaluate_thresholds(y_true, y_proba):
    thresholds = np.arange(0.0, 1.1, 0.1)
    for threshold in thresholds:
        y_pred = np.where(y_proba[:, 1] >= threshold, 1, 0)
        print(f"Threshold: {threshold:.1f}")
        print(classification_report(y_true, y_pred))
        print("--------------------------------")

In [211]:
# Визуализация ROC-кривой и выбор порога
fpr, tpr, thresholds = roc_curve(y_true, y_proba[:, 1])
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc_score(y_true, y_proba[:, 1]):.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='best')
plt.show()

In [212]:
# Визуализация Precision-Recall-кривой
precision, recall, thresholds = precision_recall_curve(y_true, y_proba[:, 1])
plt.plot(recall, precision, label='Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='best')
plt.show()

In [213]:
# Оценка метрик для различных порогов
evaluate_thresholds(y_true, y_proba)

In [214]:
# При пороге 0.5 модель дает относительно высокую точность для класса 0 (98%), а также имеет приемлемый recall (70%) для класса 1.

# Обучение модели на всех данных

In [215]:
sample_weights = compute_sample_weight(class_weight='balanced', y=y)

In [216]:
xgbc.fit(X, y, sample_weight=sample_weights)

In [217]:
# Применение лучших параметров и оценка модели
y_pred = xgbc.predict(X)
y_pred_proba = xgbc.predict_proba(X)[:, 1]
print(f"AUC-ROC: {roc_auc_score(y, y_pred_proba)}")
print(classification_report(y, y_pred))

In [218]:
def convert_dtype(dtype):
    if dtype in ['int32', 'int64', 'int16', 'int8']:
        return 'int'
    elif dtype in ['float32', 'float64']:
        return 'float'
    return dtype

# Создаем список столбцов с типами данных
columns_with_types = [f"{col}: {convert_dtype(dtype.name)}" for col, dtype in X.dtypes.items()]

# Выводим список без кавычек
for item in columns_with_types:
    print(item)

In [219]:
first_row = X.iloc[0]
columns_with_first_values = [f'"{col}": {val},' for col, val in first_row.items()]
columns_with_first_values[-1] = columns_with_first_values[-1].rstrip(',')
for item in columns_with_first_values:
    print(item)

In [220]:
import pickle

# Загрузка модели из файла
with open('G://ds-junior/final_work_ds/model/xgbc_model.pkl', 'rb') as f:
    model = pickle.load(f)

# Извлечение имен фичей
if hasattr(model, 'feature_names_in_'):
    feature_names = model.feature_names_in_
elif hasattr(model, 'get_booster'):
    # Если модель - это XGBoost
    booster = model.get_booster()
    feature_names = booster.feature_names
else:
    feature_names = None

print("Имена фичей:", feature_names)